In [0]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

 https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GSC Clustering

Based on the work of JR Oakes : https://github.com/jroakes/querycat

Before starting, you need to use the best model from huggingface.io according to your language :

https://huggingface.co/models

In [2]:
#@title Choose your model and your language

# import spacy libs
!python -m spacy download fr_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download es_core_news_sm
!python -m spacy download nl_core_news_sm
!python -m spacy download pt_core_news_sm

from IPython.display import clear_output
!git clone https://github.com/voltek62/querycat.git
!pip install -r querycat/requirements_colab.txt

TRANSFORMER_MODEL = "camembert-base" #@param {type:"string"}
RANDOM_SEED = 42
LANGUAGE = 'french' #@param {type:"string"} ["english", "german", "french", "spanish", "portuguese", "italian", "dutch"]

import querycat
querycat.init_lib(LANGUAGE)

clear_output()
print('All loaded.')

All loaded.



#### Search Console Data

##### Getting a Google API Credential.

1) Sign up for a new project in the Google APIs console at https://code.google.com/apis/console.

2) Choose "Credentials" on the left-hand menu.  

3) Choose "Create Credentials"

4) Generate an API key for your application.

5) Copy your client ID, client secret, and redirect URL into the next cell



In [0]:
#@title Configure your access
import httplib2

from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow

# Copy your credentials from the console
CLIENT_ID = 'XXXXXXXXX-5s8ll088odmhee5lusa14jhse6p4l5a4.apps.googleusercontent.com' #@param {type:"string"}
CLIENT_SECRET = 'XXXXXXXX-Oihmqnq9mPd' #@param {type:"string"}

# Check https://developers.google.com/webmaster-tools/search-console-api-original/v3/ for all available scopes
OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'

# Redirect URI for installed apps
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)

In [0]:
#@title Choose your projects

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

webmasters_service = build('webmasters', 'v3', http=http)

# Retrieve list of properties in account
site_list = webmasters_service.sites().list().execute()

# Filter for verified websites
verified_sites_urls = [s['siteUrl'] for s in site_list['siteEntry']
                       if s['permissionLevel'] != 'siteUnverifiedUser'
                          and s['siteUrl'][:4] == 'http']

# Print the URLs of all websites you are verified for.
projects = []
for site_url in verified_sites_urls:
  #print(site_url)
  projects.append(('{}'.format(site_url), site_url))

# Create Selector
import ipywidgets as widgets
output = widgets.Output()

dropdown_purpose = widgets.Dropdown(options = projects, description="Site: ")

def dropdown_project_eventhandler(change):
    output.clear_output()    
    with output:
        display(projects)

dropdown_purpose.observe(dropdown_project_eventhandler, names='value')

display(dropdown_purpose)

In [0]:
#@title Get your data
#@markdown The script cut the period into two intervals for comparing the current against the previous interval
import pandas as pd
from dateutil import rrule
from datetime import datetime, timedelta

PERIOD = 30 #@param {type:"integer"}

url = ""
if(len(dropdown_purpose.value)==0):
  print("no Site detected")
else:
  url = dropdown_purpose.value

##################
#TODO : add pagination

now = datetime.now()
before = now - timedelta(days=PERIOD)
before_date = before.strftime('%Y-%m-%d')

end_date = now - timedelta(days=1)

# init
DF = pd.DataFrame()

for dt in rrule.rrule(rrule.DAILY, dtstart=before, 
                      until=end_date):

  #if (before_date!=dt.strftime('%Y-%m-%d')):
  print(before_date+' to '+dt.strftime('%Y-%m-%d'))

  request = {
    'startDate': before_date,    
    'endDate': dt.strftime('%Y-%m-%d'),    
    'dimensions': ["date","query"], # ,"page"
    'searchType': "web",      
    'rowLimit': 25000     
  }

  response =  webmasters_service.searchanalytics().query(siteUrl=url, body=request).execute()

  dfGSC = pd.DataFrame.from_dict(response['rows'], orient='columns')

  dfGSC[["date","query"]] = pd.DataFrame(dfGSC["keys"].values.tolist()) # ,"page" 

  # drop Key column
  #print(dfGSC["keys"])
  dfGSC =  dfGSC.drop(columns=['keys'])
  DF_new = dfGSC[['date','query','clicks']]
  print(len(DF_new))

  DF = DF.append(DF_new, ignore_index=True, verify_integrity=True )
  #print(DF)

  before_date=dt.strftime('%Y-%m-%d')

In [0]:
#@title Prepare Comparative data
import numpy as np

DT = DF.copy()

DT['date'] = pd.to_datetime(DT['date'])

# cut dataset into 2 parts
interval = str(round(PERIOD/2))+"D"

# group by interval
DT = DT.groupby(['query', pd.Grouper(key='date',freq=interval)])['clicks'].sum().reset_index()

# sum by date
DT = DT.groupby(['query','date'])['clicks'].sum()
DT.columns = ['query', 'date', 'clicks']
DT = DT.reset_index()

DT = pd.pivot_table(DT, values= 'clicks', index=['query'],columns=['date'])

# replace NaN by 0
DT[np.isnan(DT)] = 0
DT = DT.reset_index()	      
DT.columns = ['query', 'previous', 'last']

# Sort by the `diff` columnn
DT.sort_values(by='last', ascending=False, inplace=True)

DT

##  Categorize the queries

The `querycat.Categorize` function, takes the following arguments.

* **Position 1**: A Pandas Dataframe that includes at least a column of search queries.
* **col**: The name of the query column from the DataFrame
* **alg**: Either `apriori` (Apriori Algorithm), or `fpgrowth` (FP Growth Algorithm)
* **min_support**: The number of times the subset terms should be found in the dataset to be included.
* **min_probablity**: FP Growth only. Finds patterns of associated terms above this minimum probability 0-1 (float)
* **min_lift**: Apriori only. Finds patterns of associated terms. <1 less probable, >1 more probable  (float)

After it runs, it will print the number of categories found, as well as the frequencies of queries in each category.

In [37]:
MINSUPPORT =  3#@param {type:"integer"}

catz = querycat.Categorize(DT, 'query', min_support=MINSUPPORT,  alg='apriori')
catz.counts.head(20)

Converting to transactions.
Normalizing the keyword corpus.
Total queries: 354
Total unique queries: 295
Total transactions: 294
Running Apriori
Making Categories
Total Categories: 133


,category
seo,75
##other##,34
nlp,13
training seo,11
data science,10
learning machine,8
course seo,7
formation data scientist,7
formation learning machine,6
data seo science,6


## Show original data
Below we print the original DataFrame with categories added.

This can be saved to CSV using `catz.df.to_csv('filename.csv', index=False)` to save and use in Excel.

In [0]:
catz.df.head(10)

## Make use of original data

This cell will aggregate data from your original CSV to produce an updated DataFrame that has current and previous period data consolidated by category.  We also include a `diff` column that shows the period change.

In [0]:
df = catz.df
df.head()

# Goup by category and aggregate the other metrics
df = df.groupby(['category'], as_index=False).agg({'query':'count', 'last':'sum', 'previous': 'sum' })

# Create a diff column
df['diff'] = df['last'] - df['previous']

# Rename the columns
df.columns = ['category', 'count of queries', 'current', 'prior', 'diff']

# Sort by the `diff` columnn
df.sort_values(by='diff', ascending=False, inplace=True)

df.head(10)

## BERT

The BERTSim class has a few functions that are useful:

* **read_df**: Reads a dataframe where the `term_col` and `diff_col` are specified as named parameters.
* **get_similar_df**: After running `read_df`, you can provide a term as a parameter to get the top matching categories, by cosine similarity.
* **diff_plot**: After running `read_df`, plots the output in 2d vector space, using either `umap`, `pca`, or `tsne` as vector reduction strategies. 

In [19]:
bsim = querycat.BERTSim(TRANSFORMER_MODEL, RANDOM_SEED)

### Read DataFrame into Class

In [0]:
bsim.read_df(df, term_col = 'category', diff_col = 'diff')

### Print similar categories to a given term.

In [0]:
df_sim = bsim.get_similar_df('train')
df_sim.sort_values(by='sim', ascending=False, inplace=True)
df_sim.head(30)

### Plot Categories

This cell plots the categories in 2D space, with reduction from BERT summed 768 dimension vectors to 2 dimension, based on the given reduction strategy.  Green is increase PoP, red is descrease.  Size is the magnitude of the change.

Reduction strategies available are `umap`, `tsne`, and `pca`



In [0]:
bsim.diff_plot(reduction='tsne')

In [0]:
bsim.diff_plot(reduction='pca')